In [36]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in the CSV
filepath = "Resources/lending_data.csv"
df=pd.read_csv(filepath)
print(df.shape)
df.head()

(77536, 8)


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


#### Check the data
---
Check to see if the data needs to be imputed, encoded, or scaled

In [3]:
# Check if data needs to be imputed or encoded
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         77536 non-null  float64
 1   interest_rate     77536 non-null  float64
 2   borrower_income   77536 non-null  int64  
 3   debt_to_income    77536 non-null  float64
 4   num_of_accounts   77536 non-null  int64  
 5   derogatory_marks  77536 non-null  int64  
 6   total_debt        77536 non-null  int64  
 7   loan_status       77536 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 4.7 MB


Data does not need to be imputed or encoded because there are no nulls and all values are numeric

In [4]:
df.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.032243
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.176646
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000


Looking at the values, we will need to do a scaler on the features. There is a lot of variations between features, so we should scale the data\
Given the number of features, we will not reduce the number of features

In [5]:
# Define the features
features = ['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt']

In [6]:
# Scaling
# subset
df_sub = df.loc[:, features]

# initialize
scaler = StandardScaler()

# fit
scaler.fit(df_sub)

# predict/transform
scaled_data = scaler.transform(df_sub)
df_scaled = pd.DataFrame(scaled_data, columns=features)

df_scaled.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,0.427304,0.426837,0.427404,0.668571,0.616143,1.043996,0.427404
1,-0.671487,-0.674918,-0.671552,-0.802166,-0.434049,-0.673973,-0.671552
2,-0.384846,-0.370249,-0.372922,-0.344424,-0.434049,-0.673973,-0.372922
3,0.427304,0.417844,0.415459,0.655346,0.616143,1.043996,0.415459
4,0.475078,0.456068,0.451295,0.694873,0.616143,1.043996,0.451295


##### Define `X` and `y` and run a `train_test_split`

In [7]:
# Get the Data
X = df_scaled
y = df.loan_status

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Review the y series and the X dataframe
display(y.head())
display(X.head())

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,0.427304,0.426837,0.427404,0.668571,0.616143,1.043996,0.427404
1,-0.671487,-0.674918,-0.671552,-0.802166,-0.434049,-0.673973,-0.671552
2,-0.384846,-0.370249,-0.372922,-0.344424,-0.434049,-0.673973,-0.372922
3,0.427304,0.417844,0.415459,0.655346,0.616143,1.043996,0.415459
4,0.475078,0.456068,0.451295,0.694873,0.616143,1.043996,0.451295


### Classificationn Model Function
---
Check to see how the following models predict the loan status

In [42]:
# Function for doing classification regressions
def classModel(model, X_train, X_test, y_train, y_test):
    # Fit the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)
    
    # Confusion Matrix
    confusion_matrix_train = confusion_matrix(y_train, train_preds)
    confusion_matrix_test = confusion_matrix(y_test, test_preds)

    tn_train = {confusion_matrix_train[0][0]}
    tp_train = {confusion_matrix_train[1][1]}
    fp_train = {confusion_matrix_train[0][1]}
    fn_train = {confusion_matrix_train[1][0]}

    # accuracy = 
    # precision = 
    # recall = 
    f1_score = (precision + recall)/2
    
    # Classification Report
    classification_report_train = classification_report(y_train, train_preds)
    classification_report_test = classification_report(y_test, test_preds)

    results = f""" Training Classification Results:
    Confusion Matrix: 
    -----------------
    {confusion_matrix_train}
    
    True Negative: {tn_train}
    True Positive: {tp_train}
    False Positive: {fp_train}
    False Negative: {fn_train}

    Accuracy: {accuracy}
    Precision: {precision}
    Recall: {recall}
    F1 Score: {f1_score}
    
    Classification Report: 
    {classification_report_train}

    -----------------------------------------------------
    
    Test Classification Results:
    Confusion Matrix: 
    -----------------
    True Negative: {confusion_matrix_test[0][0]}
    True Positive: {confusion_matrix_test[1][1]}
    False Positive: {confusion_matrix_test[0][1]}
    False Negative: {confusion_matrix_test[1][0]}
    
    Classification Report: 
    {classification_report_test}
    """

    print(results)

### Logistic Regression
---

In [41]:
# Using classModel function
# init the model
lgr = LogisticRegression(random_state=1)

classModel(lgr, X_train, X_test, y_train, y_test)

NameError: name 'accuracy' is not defined

### Additional Models
---
- Decision Tree
- Random Forest
- Support Vector Machine (SVM)
- k-Nearest Neighbor (kNN)
- Extra Trees
- Ada Boost
- Gradient Boosting
- XGBoost
- Light GBM

#### Decision Tree

#### Random Forest

#### Support Vector Machine (SVM)

#### k-Nearest Neighbor (kNN)

#### Extra Trees

#### Ada Boost

#### Gradient Boosting

#### XGBoost

#### Light GBM